In [ ]:
import os
os.chdir('../') 

In [2]:
from lib.readfile import ReadFile as rf
from lib.handle_null_values import handle_null_values as null
from lib.datetime_formatting import DatetimeFormatting as dfmt
from lib.split_time_variable import split_time_variable as split_t
from lib.correct_variable_types import correct_variable_types as vartype
from lib.handle_null_values import handle_null_values as nulval

/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
"""Read the file"""

r= rf()
df= r.read(address="lib/data/2018.csv")

In [4]:
"""Remove column with null values"""

n= null()
variables= n.delete_var_with_null_more_than(df, percentage=20)

In [5]:
"""drop unwanted variables for now"""

df2= df.drop(*variables)

In [6]:
"""select important variables""" 

important_variables=['year_built', 'bedroom_count', 'bathroom_count','lot_size', 'area', 'city_name', 'community_name', 'covered_parking', 'exterior', 'energy_source','fireplaces_count', 'floor_covering', 'garage','heating_cooling','latitude','longitude', 'property_type', 'roof', 'sold_date', 'sold_price','unfinished_sq_foot', 'year_built', 'zip_code', 'county' ]

    
important_variables= list(set(important_variables))
df3= df2.select(*important_variables)


In [7]:
"""concatenate all files"""

# files= ["09"]
# for f_no in range(10, 20):
#     files.append(f_no)    
    
#     df4= r.read(address="lib/data/20"+str(f_no)+".csv")
#     df4= df4.select(*important_variables)
#     df3 = df3.union(df4)

file= "tbl_property_details_mls_2019"
for f_no in range(1):  
    
    df4= r.read(address="lib/data/"+file+".csv")
    df4= df4.select(*important_variables)
    df3 = df3.union(df4)

In [8]:
"""filter one county from the dataset"""

df5= df3.filter( df3["county"].isin(["King"]))


In [9]:
"""correct time format"""

f= split_t()

#Assuming time variable is in timestamp type
df6= f.run(df5)


In [ ]:
"""Split time variable"""


from pyspark.sql.functions import isnan, when, count, col, from_unixtime
from pyspark.sql.types import DoubleType, TimestampType
from pyspark.sql.functions import *


df6_1= df6.withColumn('sold_date', from_unixtime(unix_timestamp('sold_date', "yyyy-MM-DD")).cast(TimestampType()))
df6_1= df6_1.withColumn('sold_date_year', year('sold_date'))
df6_1= df6_1.withColumn('sold_date_month', month('sold_date'))
df6_1= df6_1.withColumn('sold_date_day', dayofmonth('sold_date'))
# df6_1= df6_1.withColumn('sold_date_day', dayofweek('sold_date'))
df6_1= df6_1.drop('sold_date')


In [10]:
"""Correct variable types"""

v= vartype()

df7= v.run(df6)



In [11]:
"""make training and testing files and save them."""
# df_t0= df7.filter( df7["sold_date_year"]>2017)
# df_t= df_t0.filter(df_t0["sold_date_year"]<2019)
df_tst0= df7.filter( df7["sold_date_year"]==2019)
df_tst= df_tst0.filter(df7["sold_date_month"]>=8)

In [17]:
print(df_tst.count())
print(df7.count())
print(df_tst.count()-df7.count())

3355
75209
-71854


In [18]:
df_t= df7.exceptAll(df_tst)
df_t.count()


71854

In [19]:
"""duplicate 2019 data in the training file"""
df_2019= df_t.filter(df_t["sold_date_year"]==2019)
df_2019= df_2019.filter(df_2019["sold_date_year"]<8)

for i in range(2):
    df_t = df_t.union(df_2019)

In [ ]:
df7.toPandas().to_excel("lib/data/king'S_COUNTY7.0.xlsx")

In [20]:
df_train2= df_t.toPandas()
df_test2= df_tst.toPandas()

In [21]:
df_train2.to_excel("lib/data/king_train_9.xlsx")
df_test2.to_excel("lib/data/king_test_9.xlsx")

In [ ]:
# train the deep learning model and put that on the training

# def var_types(df7):
#     categorical_variables= []
#     numerical_variables=[]
#     for dt in df7.dtypes:
#         if dt[1]=="string":
#             categorical_variables.append(dt[0])
#         else:
#             numerical_variables.append(dt[0])
#     return categorical_variables, numerical_variables
        
# categorical_variables, numerical_variables= var_types(df7)

In [ ]:
# """treat null values using deep learning"""

# # add an ID in the dataset
# from pyspark.sql import functions as F
# from pyspark.sql.window import Window as W
# from pyspark import SparkContext
# from pyspark.sql import SparkSession
# from math import *


# spark_context = SparkContext.getOrCreate()
# spark = SparkSession(spark_context)

# windowSpec = W.orderBy(df7.columns[0])
# df7= df7.withColumn("id", F.row_number().over(windowSpec))
# df8= df7.createOrReplaceTempView("customer")

# #use this ID to filter the data and get results.
# base_query = "SELECT * FROM customer where "
# total_rows= df7.count()
# each_page= 500
# total_pages= ceil(total_rows/each_page)

# for p in range(total_pages):
#     query= base_query + "id > " + str(each_page*p) + " and id< " + str(each_page*(p+1))
#     df_spark= spark.sql(query)

#     df_pandas= df_spark.toPandas()
    
#     #pass this through all the models in the same order and run it.
    
    
    





# #save the model for future use.

In [22]:
# train the deep learning model and put that on the training
categorical_variables= []
numerical_variables=[]
all_variables= []
for dt in df_t.dtypes:
    if dt[1]=="string":
        categorical_variables.append(dt[0])
    else:
        numerical_variables.append(dt[0])
        
    all_variables.append(dt[0])
        
        

In [23]:
import datawig
results= {}
models={}

for c in categorical_variables: 
    var= all_variables.copy()
    var.remove(c)
    imputer = datawig.SimpleImputer(
        input_columns=var, # column(s) containing information about the column we want to impute
        output_column=c, # the column we'd like to impute values for
        output_path = 'lib/Kingimputer_models9.0/'+str(c) # stores model data and metrics
        )
    imputer.fit(train_df=df_train2, num_epochs=5)
    models[c]= imputer
    #time to test the model with entire file.  
    
#     results[c] = imputer.predict(df_pandas)
    

    

/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/pandas/core/frame.py:3137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
2019-10-17 16:28:18,434 [INFO]  
========== start: fit model
2019-10-17 16:28:18,439 [WARNING]  Already bound, ignoring bind()
2019-10-17 16:28:49,860 [INFO]  Epoch[0] Batch [0-1288]	Speed: 658.51 samples/sec	cross-entropy=1.600415	community_name-accuracy=0.509746
2019-10-17 16:29:37,319 [INFO]  Epoch[0] Train-cross-entropy=0.950970
2019-10-17 16:29:37,325 [INFO]  Epoch[0] Train-community_name-accuracy=0.670295
2019-10-17 16:29:37,326 [INFO]  Epoch[0] Time cost=78.880
2019-10-17 16:29:37,445 [INFO]  Saved checkpoint to "lib/Kingimputer_models9.0/community_name/model-0000.params"
2019

2019-10-17 16:38:51,043 [INFO]  CategoricalEncoder for column floor_covering                                found only 68 occurrences of value Ceramic Tile, Fir/Softwood, Hardwood, Vinyl, Wall to Wall Carpet
2019-10-17 16:38:51,044 [INFO]  CategoricalEncoder for column floor_covering                                found only 67 occurrences of value Laminate, Slate, Wall to Wall Carpet
2019-10-17 16:38:51,046 [INFO]  CategoricalEncoder for column floor_covering                                found only 63 occurrences of value Fir/Softwood, Vinyl
2019-10-17 16:38:51,046 [INFO]  CategoricalEncoder for column floor_covering                                found only 62 occurrences of value Ceramic Tile, Concrete, Wall to Wall Carpet
2019-10-17 16:38:51,047 [INFO]  CategoricalEncoder for column floor_covering                                found only 62 occurrences of value See Remarks, Wall to Wall Carpet
2019-10-17 16:38:51,048 [INFO]  CategoricalEncoder for column floor_covering          

2019-10-17 16:51:45,057 [INFO]  
========== start: fit model
2019-10-17 16:51:45,061 [WARNING]  Already bound, ignoring bind()
2019-10-17 16:51:55,995 [INFO]  Epoch[0] Batch [0-2021]	Speed: 2962.18 samples/sec	cross-entropy=0.121284	property_type-accuracy=0.963990
2019-10-17 16:52:09,687 [INFO]  Epoch[0] Train-cross-entropy=0.040894
2019-10-17 16:52:09,688 [INFO]  Epoch[0] Train-property_type-accuracy=0.992358
2019-10-17 16:52:09,689 [INFO]  Epoch[0] Time cost=24.625
2019-10-17 16:52:09,699 [INFO]  Saved checkpoint to "lib/Kingimputer_models9.0/property_type/model-0000.params"
2019-10-17 16:52:11,253 [INFO]  Epoch[0] Validation-cross-entropy=0.026480
2019-10-17 16:52:11,254 [INFO]  Epoch[0] Validation-property_type-accuracy=0.994306
2019-10-17 16:52:26,024 [INFO]  Epoch[1] Batch [0-2021]	Speed: 2190.67 samples/sec	cross-entropy=0.021573	property_type-accuracy=0.995611
2019-10-17 16:52:40,661 [INFO]  Epoch[1] Train-cross-entropy=0.024805
2019-10-17 16:52:40,662 [INFO]  Epoch[1] Train-pr

2019-10-17 16:54:28,925 [INFO]  CategoricalEncoder for column exterior                                found only 39 occurrences of value Cement Planked, Stucco, Wood
2019-10-17 16:54:28,927 [INFO]  CategoricalEncoder for column exterior                                found only 37 occurrences of value Cement Planked, See Remarks
2019-10-17 16:54:28,928 [INFO]  CategoricalEncoder for column exterior                                found only 36 occurrences of value Cement Planked, Cement/Concrete, Metal/Vinyl, Wood
2019-10-17 16:54:28,930 [INFO]  CategoricalEncoder for column exterior                                found only 32 occurrences of value Forced Air
2019-10-17 16:54:28,932 [INFO]  CategoricalEncoder for column exterior                                found only 32 occurrences of value Cement/Concrete, See Remarks
2019-10-17 16:54:28,933 [INFO]  CategoricalEncoder for column exterior                                found only 32 occurrences of value Cement Planked, Cement/Concret

2019-10-17 17:37:12,668 [INFO]  
========== done (2521.166447877884 s) fit model
/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/datawig/calibration.py:92: RuntimeWarning: divide by zero encountered in log
  return np.log(probas)
/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
2019-10-17 17:37:26,211 [INFO]  CategoricalEncoder for column energy_source                                found only 91 occurrences of value Electric, Oil, Wood
201

...

2019-10-17 18:10:39,938 [INFO]  
========== start: fit model
2019-10-17 18:10:39,943 [WARNING]  Already bound, ignoring bind()
2019-10-17 18:11:22,715 [INFO]  Epoch[0] Batch [0-1530]	Speed: 573.66 samples/sec	cross-entropy=1.338124	garage-accuracy=0.620183
2019-10-17 18:12:25,361 [INFO]  Epoch[0] Train-cross-entropy=1.225097
2019-10-17 18:12:25,363 [INFO]  Epoch[0] Train-garage-accuracy=0.631872
2019-10-17 18:12:25,364 [INFO]  Epoch[0] Time cost=105.415
2019-10-17 18:12:25,458 [INFO]  Saved checkpoint to "lib/Kingimputer_models9.0/garage/model-0000.params"
2019-10-17 18:12:27,500 [INFO]  Epoch[0] Validation-cross-entropy=1.242490
2019-10-17 18:12:27,503 [INFO]  Epoch[0] Validation-garage-accuracy=0.618178
2019-10-17 18:13:53,154 [INFO]  Epoch[1] Batch [0-1530]	Speed: 286.20 samples/sec	cross-entropy=1.165884	garage-accuracy=0.640349
2019-10-17 18:15:05,882 [INFO]  Epoch[1] Train-cross-entropy=1.158881
2019-10-17 18:15:05,895 [INFO]  Epoch[1] Train-garage-accuracy=0.639643
2019-10-17 18

2019-10-17 18:25:41,256 [INFO]  CategoricalEncoder for column heating_cooling                                found only 79 occurrences of value Central A/C, Forced Air, Insert
2019-10-17 18:25:41,258 [INFO]  CategoricalEncoder for column heating_cooling                                found only 77 occurrences of value None
2019-10-17 18:25:41,260 [INFO]  CategoricalEncoder for column heating_cooling                                found only 76 occurrences of value Tankless Water Heater
2019-10-17 18:25:41,262 [INFO]  CategoricalEncoder for column heating_cooling                                found only 76 occurrences of value Central A/C, Forced Air, Hot Water Recirc Pump
2019-10-17 18:25:41,264 [INFO]  CategoricalEncoder for column heating_cooling                                found only 75 occurrences of value Radiant, Ductless HP-Mini Split, Tankless Water Heater
2019-10-17 18:25:41,266 [INFO]  CategoricalEncoder for column heating_cooling                                found only

2019-10-17 18:25:41,345 [INFO]  CategoricalEncoder for column heating_cooling                                found only 39 occurrences of value Central A/C, Forced Air, Heat Pump, Stove/Free Standing
/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/pandas/core/frame.py:3137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
2019-10-17 18:26:16,518 [INFO]  
========== start: fit model
2019-10-17 18:26:16,523 [WARNING]  Already bound, ignoring bind()
2019-10-17 18:27:11,331 [INFO]  Epoch[0] Batch [0-1809]	Speed: 528.85 samples/sec	cross-entropy=2.207044	heating_cooling-accuracy=0.457182
2019-10-17 18:28:39,364 [INFO]  Epoch[0] Train-cross-entropy=2.004926
2019-10-17 18:28:39,365 [INFO]  Epoch[0] Train-heating

2019-10-17 21:40:14,272 [INFO]  CategoricalEncoder for column roof                                found only 14 occurrences of value Flat, See Remarks, Torch Down
2019-10-17 21:40:14,275 [INFO]  CategoricalEncoder for column roof                                found only 13 occurrences of value Flat, Tile
2019-10-17 21:40:14,279 [INFO]  CategoricalEncoder for column roof                                found only 13 occurrences of value Green (Living)
2019-10-17 21:40:14,283 [INFO]  CategoricalEncoder for column roof                                found only 12 occurrences of value Composition, Flat, Metal
2019-10-17 21:40:14,285 [INFO]  CategoricalEncoder for column roof                                found only 12 occurrences of value Built-Up, Composition, Torch Down
2019-10-17 21:40:14,287 [INFO]  CategoricalEncoder for column roof                                found only 11 occurrences of value Closing, Negotiable
2019-10-17 21:40:14,292 [INFO]  CategoricalEncoder for column roof 

2019-10-17 21:40:14,420 [INFO]  CategoricalEncoder for column roof                                found only 2 occurrences of value 699950.00
2019-10-17 21:40:14,423 [INFO]  CategoricalEncoder for column roof                                found only 2 occurrences of value 1315000.00
2019-10-17 21:40:14,425 [INFO]  CategoricalEncoder for column roof                                found only 2 occurrences of value 775000.00
2019-10-17 21:40:14,428 [INFO]  CategoricalEncoder for column roof                                found only 2 occurrences of value 1289950.00
/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/pandas/core/frame.py:3137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]
2019-10-17 21:40:59,

In [24]:
xyz= df_test2
for c in categorical_variables:
    m= models[c]
    xyz= m.predict(xyz)
    


xyz2= df_train2
for c in categorical_variables:
    m= models[c]
    xyz2= m.predict(xyz2)


/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/datawig/calibration.py:92: RuntimeWarning: divide by zero encountered in log
  return np.log(probas)
/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/datawig/calibration.py:92: RuntimeWarning: divide by zero encountered in log
  return np.log(probas)
/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/datawig/calibration.py:92: RuntimeWarning: divide by zero encountered in log
  return np.log(probas)
/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/datawig/calibration.py:92: RuntimeWarning: divide by zero encountered in log
  return np.log(probas)
/Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/datawig/calibration.py:92: RuntimeWarning: divide by zero encountered in log
  return np.log(probas)
/Users/muddassarsharif/Desktop/muddassar/productio

In [25]:
def delete_extra_categorical_variables(df=None, variables=None):
    if variables==None:
        varaibles= self.categorical_variables
        
        #make a list of all the categorical columns. 
    no_n=[]   

        #delete all unnecessary columns, including the one that make no senese.
    for cv in variables:
        no_n.append(cv+'_imputed_proba')
        no_n.append(cv)

    df= df.drop(no_n, axis=1)
        
    return df


xyz111= delete_extra_categorical_variables(xyz, categorical_variables)


In [26]:
#Run h2o library offfline
xyz222= delete_extra_categorical_variables(xyz2, categorical_variables)

In [27]:
xyz111.to_excel("lib/data/king_train_9.1.xlsx")
xyz222.to_excel("lib/data/king_test_9.1.xlsx")

In [37]:
# xyz111.to_parquet("lib/data/King'S_COUNTY_test7.0.parquet",engine='fastparquet', compression='gzip'))
# # xyz222.to_excel("lib/data/King'S_COUNTY_train7.0.xlsx")

from fastparquet import write 
write("lib/data/king_test_9.1.parquet", xyz111)
write("lib/data/king_train_9.1.xlsx", xyz222)

ModuleNotFoundError: No module named 'fastparquet'

In [ ]:
xyz222.to_parquet("lib/data/King'S_COUNTY_train7.0.xlsx",engine='fastparquet', compression='gzip')

In [ ]:
write("lib/data/King'S_COUNTY_train7.1.parquet", xyz222)

In [39]:
!pip install fastparquet

  Using cached https://files.pythonhosted.org/packages/58/49/dccb790fa17ab3fbf84a6b848050083c7a1899e9586000e34e3e4fbf5538/fastparquet-0.3.2.tar.gz
  Running setup.py bdist_wheel for fastparquet ... error
  Complete output from command /Users/muddassarsharif/Desktop/muddassar/production/API/env4/bin/python3 -u -c "import setuptools, tokenize;__file__='/private/var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/pip-install-qb287m4g/fastparquet/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /private/var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/pip-wheel-p_vukt_0 --python-tag cp36:
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.9-x86_64-3.6
  creating build/lib.macosx-10.9-x86_64-3.6/fastparquet
  copying fastparquet/encoding.py -> build/lib.macosx-10.9-x86_64-3.6/fastparquet
  copying fastparquet/util.py -> build/lib.macosx-10

Command "/Users/muddassarsharif/Desktop/muddassar/production/API/env4/bin/python3 -u -c "import setuptools, tokenize;__file__='/private/var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/pip-install-qb287m4g/fastparquet/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /private/var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/pip-record-5ez9s2nt/install-record.txt --single-version-externally-managed --compile --install-headers /Users/muddassarsharif/Desktop/muddassar/production/API/env4/include/site/python3.6/fastparquet" failed with error code 1 in /private/var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/pip-install-qb287m4g/fastparquet/
You are using pip version 18.1, however version 19.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [32]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd


In [33]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "1.8.0_191"; Java(TM) SE Runtime Environment (build 1.8.0_191-b12); Java HotSpot(TM) 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /Users/muddassarsharif/Desktop/muddassar/production/API/env4/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/tmpllf2lbbd
  JVM stdout: /var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/tmpllf2lbbd/h2o_muddassarsharif_started_from_python.out
  JVM stderr: /var/folders/2h/ylrmltv95sd1_w40dsmy51m00000gn/T/tmpllf2lbbd/h2o_muddassarsharif_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.6
H2O cluster version age:,15 days
H2O cluster name:,H2O_from_python_muddassarsharif_h7enal
H2O cluster total nodes:,1
H2O cluster free memory:,1.778 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"accepting new members, healthy"


In [34]:
# train = h2o.import_file("lib/data/Snohomish'S_COUNTY_train6.3.csv")
# test = h2o.import_file("lib/data/Snohomish'S_COUNTY_test6.3.csv")

train = h2o.H2OFrame(xyz222)
test = h2o.H2OFrame(xyz111)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [35]:
# Identify predictors and response
x = train.columns
y = 'sold_price'
x.remove(y)

In [42]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=50, seed=200)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [43]:
preds = aml.leader.predict(test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [44]:
p= preds.as_data_frame()

In [45]:
import numpy as np
lst_p = list(np.array(p['predict']))
lst_a = list(np.array(test.as_data_frame()['sold_price']))

new_lst = []
mape= []
for i in range(len(lst_p)):
    
    mape.append(float(float(((lst_p[i]-lst_a[i])**2)**(1/2))/lst_a[i]))
    new_lst.append(float(lst_p[i]))


In [49]:
re= test.as_data_frame()

In [47]:
re["sold_price_predict"]= new_lst
re["APE"]= mape

In [48]:
re.to_excel("lib/data/POC-draft-King-9.0.xlsx")

In [ ]:
model_path = h2o.save_model(model=aml.leader, path="lib/data/mymodel-King9.0", force=True)

In [ ]:
df7.dtypes

In [ ]:
df7.write.parquet("lib/data/df7.parquet")

In [ ]:
# Read in the Parquet file created above.
# Parquet files are self-describing so the schema is preserved.
# The result of loading a parquet file is also a DataFrame.

# imports
from pyspark import SparkContext
from pyspark.sql import SparkSession
from lib.logs import logger

spark_context = SparkContext.getOrCreate()
spark = SparkSession(spark_context)
df71 = spark.read.parquet("lib/data/df7.parquet")

In [ ]:
df71.dtypes

In [ ]:
xyz111.count()

In [ ]:
from tpot import TPOTRegressor
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

housing = load_boston()
X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target,
                                                    train_size=0.75, test_size=0.25)

